# Makemore

A character-level language model that is learnt in an auto-regressive fashion.    

Becoming a backprop ninja, swole doge style.

In [1]:
import torch
import torch.nn.functional as F
import matplotlib.pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set()

SEED = 2147483647

In [2]:
# read in all the words
words = open('./data/names.txt', 'r').read().splitlines()
print(f"A few words: {words[:5]}")
print(f"Length of words: {len(words)}")


A few words: ['emma', 'olivia', 'ava', 'isabella', 'sophia']
Length of words: 32033


In [3]:
# Build the vocabulary of characters and mappings to/from integers.
chars = sorted(list(set(''.join(words))))
stoi = {s:i+1 for i,s in enumerate(chars)}
stoi['.'] = 0
itos = {i:s for s,i in stoi.items()}
vocab_size = len(itos)
print(itos)
print(vocab_size)

{1: 'a', 2: 'b', 3: 'c', 4: 'd', 5: 'e', 6: 'f', 7: 'g', 8: 'h', 9: 'i', 10: 'j', 11: 'k', 12: 'l', 13: 'm', 14: 'n', 15: 'o', 16: 'p', 17: 'q', 18: 'r', 19: 's', 20: 't', 21: 'u', 22: 'v', 23: 'w', 24: 'x', 25: 'y', 26: 'z', 0: '.'}
27
